In [30]:
import os
import copy 
import pandas as pd
import sys
mport cobra

SyntaxError: invalid syntax (329256765.py, line 5)

In [10]:
#Load Model
mainPATH = '/Users/ivanlizat/Documents/devel/folderForGit/PseudomonasPutidaCobraPy'
relativModelPATH = mainPATH + '/MatlabExportedModels'
''' 
iJN1462_GLC_UR10_9_main     =   cobra.io.read_sbml_model( relativModelPATH + '/iJN1462_GLC_UR10_9.xml')
iJN1462_GLC_UR6_3_main      =   cobra.io.read_sbml_model( relativModelPATH + '/iJN1462_GLC_UR6_3.xml')
iJN1462_GLC_UR7_3_main      =   cobra.io.read_sbml_model( relativModelPATH + '/iJN1462_GLC_UR7_3.xml')
'''

iJN1462_OCT_UR3_4_main      =   cobra.io.read_sbml_model(relativModelPATH + '/iJN1462_OCT_UR3_4.xml')
'''
iJN1462_GLC_UR10_9          =   copy.deepcopy(iJN1462_GLC_UR10_9_main)
iJN1462_GLC_UR6_3           =   copy.deepcopy(iJN1462_GLC_UR6_3_main)
iJN1462_GLC_UR7_3           =   copy.deepcopy(iJN1462_GLC_UR7_3_main)
iJN1462_GLN_UR5_1           =   copy.deepcopy(iJN1462_GLN_UR5_1_main)
'''
iJN1462_OCT_UR3_4           =   copy.deepcopy(iJN1462_OCT_UR3_4_main)

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [15]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [28]:
from cobrapy.src import cobra
#das gesamte Paket als 
df = iJN1462_OCT_UR3_4.summary().to_frame()
print(type(df))

<class 'pandas.core.frame.DataFrame'>
